In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from pandas.io.json import json_normalize
import pandas as pd
import numpy as np


import re 

from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

import sympound 
from autocorrect import Speller
import splitter

from spellchecker import SpellChecker

import gensim
import nltk
from nltk.corpus import stopwords


Using TensorFlow backend.
/home/thanish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/thanish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/thanish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/thanish/.local/lib/python3.6/site-packages/tensorflow/py

ModuleNotFoundError: No module named 'sympound'

In [ ]:
import os 
os.getcwd()

In [2]:
# Reading the processed text

train_DF = pd.read_csv('../data/train_corrected.csv')
test_DF  = pd.read_csv('../data/test_corrected.csv')

In [3]:
# Convert the label to OHE
train_DF = pd.concat([train_DF[['ID', 'text']], pd.get_dummies(train_DF.label)], axis = 1)
labels = ['Alcohol', 'Depression', 'Drugs', 'Suicide']
train_DF

,ID,text,Alcohol,Depression,Drugs,Suicide
0,SUAVK39Z,i feel that it was better i dream happy,0,1,0,0
1,9JDAGUV3,why do i get hallucinations,0,0,1,0
2,419WR1LQ,i am stressed due to lack of financial support...,0,1,0,0
3,6UY7DX6Q,why is life important,0,0,0,1
4,FYC0FTFB,how could i be helped to go through the depres...,0,1,0,0
...,...,...,...,...,...,...
611,BOHSNXCN,what should i do to stop alcoholism,1,0,0,0
612,GVDXRQPY,how to become my oneself again,0,0,0,1
613,IO4JHIQS,how can someone stop it,1,0,0,0
614,1DS3P1XO,i feel unworthy,0,1,0,0


In [14]:
MAX_words_2_use = 50000
max_seq_len = 196
Embedding_dim = 50

In [15]:
tokenizer = Tokenizer(num_words=MAX_words_2_use, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(np.append(train_DF.text.values,  test_DF.text.values))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1003 unique tokens.


In [16]:
train_prod_DF = tokenizer.texts_to_sequences(train_DF['text'].values)
test_prod_DF = tokenizer.texts_to_sequences(test_DF['text'].values)

train_content_embed = pad_sequences(train_prod_DF, maxlen=max_seq_len, padding='post')
print('Shape of train data tensor:', train_content_embed.shape)

test_content_embed = pad_sequences(test_prod_DF, maxlen=max_seq_len, padding='post')
print('Shape of test data tensor:', test_content_embed.shape)

Shape of train data tensor: (616, 196)
Shape of test data tensor: (309, 196)


In [17]:
train_content_embed_DF = pd.DataFrame(train_content_embed)
train_content_embed_DF.columns = ['embed' + "_" + str(x) for x in range(0, max_seq_len)]
train_content_embed_DF[labels] = train_DF[labels]
train_content_embed_DF['ID'] = train_DF.ID
print("Content Embed shape",train_content_embed_DF.shape)

train_content_embed_DF.head()

Content Embed shape (616, 201)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_191,embed_192,embed_193,embed_194,embed_195,Alcohol,Depression,Drugs,Suicide,ID
0,1,4,43,23,26,47,1,502,188,0,...,0,0,0,0,0,0,1,0,0,SUAVK39Z
1,21,13,1,32,343,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,9JDAGUV3
2,1,16,70,268,2,105,5,94,189,19,...,0,0,0,0,0,0,1,0,0,419WR1LQ
3,21,12,15,230,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6UY7DX6Q
4,3,71,1,37,190,2,121,133,6,17,...,0,0,0,0,0,0,1,0,0,FYC0FTFB


In [18]:
test_content_embed_DF = pd.DataFrame(test_content_embed)
test_content_embed_DF.columns = ['embed' + "_" + str(x) for x in range(0, max_seq_len)]
test_content_embed_DF['ID'] = test_DF.ID
print("Content Embed shape", test_content_embed_DF.shape)

test_content_embed_DF.head()

Content Embed shape (309, 197)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_187,embed_188,embed_189,embed_190,embed_191,embed_192,embed_193,embed_194,embed_195,ID
0,3,2,31,51,109,7,422,0,0,0,...,0,0,0,0,0,0,0,0,0,02V56KMO
1,1,4,22,128,59,19,15,0,0,0,...,0,0,0,0,0,0,0,0,0,03BMGTOK
2,1,26,34,30,4,22,209,64,481,2,...,0,0,0,0,0,0,0,0,0,03LZVFM6
3,1,4,34,41,411,259,1,82,64,38,...,0,0,0,0,0,0,0,0,0,0EPULUM5
4,8,1,37,831,53,1,16,14,170,325,...,0,0,0,0,0,0,0,0,0,0GM4C5GD


In [19]:
train_content_embed_DF.shape, test_content_embed_DF.shape

((616, 201), (309, 197))

In [20]:
test_content_embed

array([[  3,   2,  31, ...,   0,   0,   0],
       [  1,   4,  22, ...,   0,   0,   0],
       [  1,  26,  34, ...,   0,   0,   0],
       ...,
       [  3,   8,   1, ...,   0,   0,   0],
       [ 12,  23, 465, ...,   0,   0,   0],
       [  3,   8,   1, ...,   0,   0,   0]], dtype=int32)

In [21]:
dep = labels
indep = train_content_embed_DF.columns.difference(labels + ['ID'])

np.random.seed(100)
train_local_X, valid_local_X, train_local_Y, valid_local_Y = train_test_split(train_content_embed,
                                                                              train_DF[dep],
                                                                              test_size = 0.2,
                                                                              random_state = 100)
print(train_local_X.shape, valid_local_X.shape, train_local_Y.shape, valid_local_Y.shape)

(492, 196) (124, 196) (492, 4) (124, 4)


In [29]:
model = Sequential()
model.add(Embedding(MAX_words_2_use, Embedding_dim, input_length= train_local_X.shape[1]))
#model.add(SpatialDropout1D(0.2))
#model.add(LSTM(10, dropout = 0.2))
model.add(LSTM(100, return_sequences = True ))
model.add(LSTM(100))
model.add(Dense(len(dep), activation = 'sigmoid'))

model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', 'binary_crossentropy'])


In [30]:
epochs = 10
batch_size = 16

early_stop = EarlyStopping(monitor='val_loss', 
                           verbose = True,
                           mode = 'min',
                           patience=3, 
                           min_delta=0.0001)

history = model.fit(train_local_X, train_local_Y, 
                    epochs=epochs, 
                    batch_size= batch_size,
                    validation_data= (valid_local_X, valid_local_Y),
                    callbacks=[
                        #early_stop
                    ])

/home/thanish/.local/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:414: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 492 samples, validate on 124 samples
Epoch 1/10
492/492 [==============================] - 8s 16ms/step - loss: 0.5237 - accuracy: 0.7642 - binary_crossentropy: 0.5237 - val_loss: 0.4777 - val_accuracy: 0.7823 - val_binary_crossentropy: 0.4777
Epoch 2/10
492/492 [==============================] - 6s 13ms/step - loss: 0.4724 - accuracy: 0.7866 - binary_crossentropy: 0.4724 - val_loss: 0.4778 - val_accuracy: 0.7823 - val_binary_crossentropy: 0.4778
Epoch 3/10
492/492 [==============================] - 6s 13ms/step - loss: 0.4712 - accuracy: 0.7795 - binary_crossentropy: 0.4712 - val_loss: 0.4782 - val_accuracy: 0.7500 - val_binary_crossentropy: 0.4782
Epoch 4/10
492/492 [==============================] - 6s 13ms/step - loss: 0.4701 - accuracy: 0.7876 - binary_crossentropy: 0.4701 - val_loss: 0.4740 - val_accuracy: 0.7823 - val_binary_crossentropy: 0.4740
Epoch 5/10
492/492 [==============================] - 6s 13ms/step - loss: 0.4694 - accuracy: 0.7866 - binary_crossentropy: 0.

In [28]:
model.predict(valid_local_X)

array([[0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.2357306 , 0.5537685 , 0.10428799, 0.10621298],
       [0.23573056, 0.5537684 , 0.10428798, 0.10621296],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.2357306 , 0.5537685 , 0.10428799, 0.10621298],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573056, 0.5537684 , 0.10428798, 0.10621296],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.2357306 , 0.5537685 , 0.10428799, 0.10621298],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.23573057, 0.55376846, 0.10428799, 0.10621297],
       [0.2357306 , 0.5537685 , 0.10428799, 0.10621298],
       [0.23573057, 0.55376846,

In [ ]:
final_output = final_output[['ID', 'Depression', 'Alcohol', 'Suicide', 'Drugs']]
final_output.to_csv('../output/sub_25_XGBoost.csv', index = False)